# Covid Tweet Sentiment Classifier

Write a description of this project

## 0. Imports

Include discussion about file structure and choice of machine learning frameworks (for us, it will be PyTorch)

In [17]:
import numpy as np
import pandas as pd
import engine
import tweepy

print("Imports succeeded")
#engine.sanity_check()

Imports succeeded


## 1. Data Processing

We need to perform the following steps:
- [x] Read the raw data
- [X] Scrape the tweets using the IDs
- [ ] Bin the annotations into either legit information or misinformation

Ideally, we create a new dataset .csv and save it. It should have the following headers: "id", "text", "annotation1"


In [2]:
raw_data = pd.read_csv('../data/miscov19.csv')
raw_data.tail()

,status_id,status_created_at,annotation1,annotation2,text
4568,1280654762950774785,Wed Jul 08 00:07:43 +0000 2020,calling out or correction,NaN,Da fark is wrong with these people?!\n\nhttps:...
4569,1280669090835828738,Wed Jul 08 01:04:39 +0000 2020,fake cure,NaN,Doctor Finds Cure to COVID-19 (without Vaccine...
4570,1280675380014309376,Wed Jul 08 01:29:39 +0000 2020,calling out or correction,NaN,I can’t believe people legitimately think that...
4571,1280676605396836352,Wed Jul 08 01:34:31 +0000 2020,politics,NaN,"...(wrong with Trump continues), \n-Will push ..."
4572,1280693005599506432,Wed Jul 08 02:39:41 +0000 2020,calling out or correction,NaN,For anyone who still believes that COVID-19 is...


Here, we try to take the tweet IDs and obtain the text from them.

## 2. Model Training

In this section, we will train the model. The model will be defined in the engine.py file along with data loading and associated functions. We will need to develop an API to go between this Jupyter Notebook and the engine. This layer of separation is intentional, as it allows us to have pipeline components which are self contained.

### 2.1 Hyperparameter Optimization 

Using K-fold validation, we shall attempt to find optimal hyperparameters. The hyperparameter space has yet to be defined, but early stopping is known to be an important aspect of fine tuning BERT models.

## 3. Model Evaluation

In this section, we will evaluate the model based on its performance on a holdout set of test data.

In [16]:
import model_evaluate

#model_evaluate.sanity_check()

X_test = [0,0,0,0,0,0,0,0,0,0,0] # list-like of test data
y_real = [1,0,0,1,1,0,0,1,0,1,0] # list-like of real labels on the test data
y_pred = [1,0,1,1,0,0,0,1,1,0,0] # list-like of predicted labels on the test data

acc = model_evaluate.accuracy(y_real, y_pred)
pre = model_evaluate.precision(y_real, y_pred)
rec = model_evaluate.recall(y_real, y_pred)
f1 = model_evaluate.f1(y_real, y_pred)

print(f"Accuracy: {acc:.4f}\nPrecision: {pre:.4f}\nRecall: {rec:.4f}\nF1: {f1:.4f}")

Accuracy: 0.6364
Precision: 0.6000
Recall: 0.6000
F1: 0.6000
